# KMNIST Example with your own implementation (BONUS)

Now, in this part, you will work with Kuzushiji-MNIST data (https://github.com/rois-codh/kmnist) for character classification. 
The images contain one of the 10 characters in Kuzusjihi(cursive Japanese) Alphabet.
Use an appropriate loss function.

You should build a ConvNet architecture including all layers such as Conv2d, Maxpool, Dropout, and BatchNorm. You are free to design the layers as you like.

IMPORTANT: You are NOT allowed to use sklearn or any other implementations for the learning part . You are ALLOWED ONLY TO USE your own implementation from the above steps.

"KMNIST Dataset" (created by CODH), adapted from "Kuzushiji Dataset" (created by NIJL and others), doi:10.20676/00000341

In [1]:
from blg561.layer import layers_with_weights
from blg561.layer import layer
from blg561.layer.optimizers import SGDWithMomentum, VanillaSDGOptimizer
from blg561.layer.model import Model
from blg561.checks import grad_check,rel_error
import numpy as np
import scipy as sci
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
# create your own dataloader
kmnist_train_imgs_path = "kmnist/kmnist-train-imgs.npz"
kmnist_train_labels_path = "kmnist/kmnist-train-labels.npz"
kmnist_test_imgs_path = "kmnist/kmnist-test-imgs.npz"
kmnist_test_labels_path = "kmnist/kmnist-test-labels.npz"

def create_permutation(x, y):
    perm = np.random.permutation(len(x))
    return x[perm], y[perm]

def load_kmnist():
    train = np.load(kmnist_train_imgs_path)
    train_imgs = train.f.arr_0
    train_labels = np.load(kmnist_train_labels_path)
    train_labels = train_labels.f.arr_0
    test = np.load(kmnist_test_imgs_path)
    test_imgs = test.f.arr_0
    test_labels = np.load(kmnist_test_labels_path)
    test_labels = test_labels.f.arr_0
    train_imgs, train_labels = create_permutation(train_imgs,train_labels)
    train_imgs = np.expand_dims(train_imgs, axis=1)
    train_labels = np.expand_dims(train_labels, axis=1)
    test_imgs = np.expand_dims(test_imgs, axis=1)
    test_labels = np.expand_dims(test_labels, axis=1)
    
    return train_imgs,train_labels,test_imgs,test_labels



#### Create the model

In below, we provide an example model.

In [33]:
model = Model()
layers = [
          layers_with_weights.Conv2d(in_size=1, out_size=3, kernel_size=5, stride=1, padding=1),
          layer.ReLU(), 
          layers_with_weights.Conv2d(in_size=3, out_size=25, kernel_size=5, stride=2, padding=1),
          layer.ReLU(), 
          layers_with_weights.Conv2d(in_size=25, out_size=50, kernel_size=5, stride=2, padding=1),
          layer.ReLU(), 
          layers_with_weights.Conv2d(in_size=50, out_size=50, kernel_size=5, stride=2, padding=1),
          layer.Flatten(), 
          layers_with_weights.AffineLayer(200, 64), 
          layer.ReLU(),
          layers_with_weights.AffineLayer(64,10),
          layer.Softmax()
        ]
model(layers) # Load layers to model object


#### Train your model with the data and show the results as Loss Curves and Accuracy for Test in a Confusion Matrix (You can use scikit-learn's confusion matrix)

In [37]:
train_accs = []
test_accs = []
train_losses = []
test_losses = []

# Options
shuffle_on_each_epoch = True
regularization_strength = 0
n_epochs = 100
print_every = 1
test_every = 100
train_imgs,train_labels,test_imgs,test_labels = load_kmnist()

optimizer = SGDWithMomentum(model,lr=1e-1, regularization_str=regularization_strength)
#try with vanilla optimizer
#optimizer = layer.VanillaSDGOptimizer(model,lr=1e-1, regularization_str=regularization_strength)

for epoch in range(n_epochs):
    softmax_out = model.forward(train_imgs[:1000])
    
    predictions = np.argmax(softmax_out, axis=1)
    train_acc = np.mean(predictions == train_labels[:1000])
    loss = layer.loss(softmax_out, train_labels[:1000])
    
    train_accs.append(train_acc)
    train_losses.append(loss)
    if epoch % print_every == 0:
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, loss, train_acc))
    
    model.backward(train_labels[:1000])
    optimizer.optimize()
    
    if epoch % test_every == 0:
        softmax_out = model.forward(test_imgs[:1000])
        predictions = np.argmax(softmax_out, axis=1)
        loss = layer.loss(softmax_out, test_labels[:1000])
        test_acc = np.mean(predictions == test_labels[:1000])
        test_losses.append([loss for _ in range(test_every)])
        test_accs.append([test_acc for _ in range(test_every)])
        print("Epoch: {}, Test Loss: {}, Test Accuracy: {}".format(epoch, loss, test_acc))

Epoch: 0, Loss: nan, Accuracy: 0.105
Epoch: 0, Test Loss: nan, Test Accuracy: 0.105
Epoch: 1, Loss: nan, Accuracy: 0.105
Epoch: 2, Loss: nan, Accuracy: 0.105


KeyboardInterrupt: 